In [1]:
import arcpy
import datetime
import numpy as np
import pandas as pd

from datetime import datetime
from datetime import timedelta

In [25]:
now = datetime.now()
wksp = 'C:/Users/friendde/Documents/ArcGIS/Projects/VegetationManagement/VegetationManagement.gdb'
trim_polygons = 'C:/Users/friendde/Documents/ArcGIS/Projects/VegetationManagement/VegetationManagement.gdb/TrimPolygons'
# using Query https://developers.arcgis.com/rest/services-reference/query-feature-service-layer-.htm
# GeoNet post https://community.esri.com/thread/165713
#trim_polygons = 'https://services3.arcgis.com/w0ynEuE1ne6vjfu0/arcgis/rest/services/IlluminateVegDemo/FeatureServer/0/query'
trim_fields = ['TrimStatus','ScheduleIVR','GlobalID']
trim_clause = "TrimStatus = 'CrewAssigned'"

transformer_bank= 'C:/GISData/Data/Snapshot/mxElectric.geodatabase/Electric/main.eTransformerBank'
# transformer bank is related to service point using 'eTransformerBank_GLOBALID'
service_points = 'C:/GISData/Data/Snapshot/mxElectric.geodatabase/Electric/main.eServicePoint'
service_points_fields = ['GlobalID','eTransformerBank_GLOBALID']
# service point is related to eCustomerAccount using eServicePoint_GLOBALID which stores the SAP Installation Number
ecustomer_account = 'C:/GISData/Data/Snapshot/mxElectric.geodatabase/main.eCustomerAccount'
ecustomer_account_fields = ['eServicePoint_GLOBALID','Install_Num']
# eCustomer Account is related to SAP_Installaion table using Install_Num.
# SAP_Installation holds all customer data
sap_table = 'C:/GISData/Data/Snapshot/mxElectric.geodatabase/main.SAP_Installation'
ivr_list = 'C:/Users/friendde/Documents/ArcGIS/Projects/VegetationManagement/IVRList.csv'

arcpy.env.workspace = wksp
arcpy.env.overwriteOutput = True

In [33]:
trim_GUIDS = []
with arcpy.da.SearchCursor(trim_polygons,trim_fields,"TrimStatus = 'CrewAssigned' AND ScheduleIVR IS NOT NULL") as sc:
    for row in sc:
        ivr_date = row[1]
        if ivr_date <= now+timedelta(days=14):
            trim_GUIDS.append(row[2])

GUIDList = ','.join(f"'{guids}'" for guids in trim_GUIDS)
expression = 'GlobalID IN (' + GUIDList +')'

In [34]:
expression

"GlobalID IN ('{0C67085A-7E3D-4B5D-BE47-F21C23E2978E}','{21102F32-3A1E-4777-B0D8-4590A1E50AF3}','{68203C4F-A11E-4270-AC34-8C5634A0B998}','{115D18C4-A534-4A74-A76E-2AECE3329A8A}','{3AEA2E13-721F-4FBE-AC9E-C0BCDC04BA6F}','{98AAB139-7F86-4A39-8279-D946DE0EE7BD}','{80AEAFB5-7A58-400E-B3BE-760068E8F1AC}','{01C29274-9A79-45FF-9623-B5CA7EA1DD10}','{FC9C2686-3456-4DA3-B0A0-55F696DC5DDE}','{DA176222-280C-405D-BB15-09050EF0FF8A}','{10BB287F-BA0B-4654-9AD3-9CE0C1C50F93}','{B62C244F-62DB-4E12-9B17-4C876E5F928A}','{46998B84-03FA-4422-B9D7-8A394E9522CC}','{44A0F728-C671-47C7-BEAF-5A5A1A103F25}','{0FFDD848-CA77-4CD4-BA95-94126EF44E70}','{9980DBB2-3B71-411D-9B6E-1FD3A53EA06B}','{108C4D2F-2819-43D8-9A5D-AFDC1037711A}','{DE708A9F-3B1F-426A-A195-E34DF37890A8}','{0B3C43FB-A76A-4464-B650-DF0B82C40AFE}','{4A7DE020-5B91-4A92-AB72-A855CC396318}','{164C5F5F-DDBC-4203-8B35-727785E3F2E8}','{56E4F7E1-AA51-4116-8AF6-261718F3B2C0}','{7A363964-B16B-4681-92E3-B9982D9DA35B}','{83B8E353-04A9-4A10-85B2-1AA8011FFD32}','{

In [26]:
arcpy.MakeFeatureLayer_management(trim_polygons,'trim_polygons_flayer',expression)

result = arcpy.GetCount_management('trim_polygons_flayer')
print(f'Trim Polygons Feature layer Count {int(result.getOutput(0))}')

Trim Polygons Feature layer Count 131


In [44]:
arcpy.MakeFeatureLayer_management(transformer_bank,'transformer_bank_flayer')

result = arcpy.GetCount_management('transformer_bank_flayer')
print(f'Transformer Bank Feature layer Count {int(result.getOutput(0))}')

TX Feature layer Count 16509


In [35]:
arcpy.SelectLayerByLocation_management('transformer_bank_flayer', 'WITHIN', 'trim_polygons_flayer')

result = arcpy.GetCount_management('transformer_bank_flayer')
print(f'Count of Transformer Banks Within Trim Polygon Feature Layer {int(result.getOutput(0))}')

TX By Location Count 20


In [36]:
arcpy.MakeFeatureLayer_management('transformer_bank_flayer','selectedTX')

result = arcpy.GetCount_management('selectedTX')
print(f'Count of Selected Transformer Feature layer {int(result.getOutput(0))}')

TX Feature layer Count 20


In [37]:
txGUIDs = []
with arcpy.da.SearchCursor('selectedTX',['GlobalID']) as txcur:
    for txrow in txcur:
        txGUIDs.append(txrow[0])


GUIDList = ','.join(f"'{guids}'" for guids in txGUIDs)
expression = f'eTransformerBank_GLOBALID IN ({GUIDList})'

In [38]:
svcpntGUIDS = []
with arcpy.da.SearchCursor(service_points,service_points_fields,expression) as svccur:
    for svcrow in svccur:
        svcpntGUIDS.append(svcrow[0])

#result = arcpy.GetCount_management('Service Point')
#print(f'Service Point Count {int(result.getOutput(0))}')

GUIDList = ','.join(f"'{guids}'" for guids in svcpntGUIDS)
expression = f'eServicePoint_GLOBALID IN ({GUIDList})'

In [39]:
installNums = []
with arcpy.da.SearchCursor(ecustomer_account,ecustomer_account_fields,expression) as custcur:
    for custrow in custcur:
        installNums.append(custrow[1])

CustomerList = ','.join(f"'{guids}'" for guids in installNums)
expression = f'Install_Num IN ({CustomerList})'

In [40]:
arcpy.MakeTableView_management(sap_table,'IVRList',expression)

result = arcpy.GetCount_management('IVRList')
print(f'Customer IVR List Count {int(result.getOutput(0))}')

Customer Count 158


In [41]:
nparr= arcpy.da.TableToNumPyArray('IVRList','*')
df = pd.DataFrame(nparr)

In [42]:
df.head()

,OBJECTID,INSTALL_NUM,INSTALL_TYPE,DIVISION,CUST_NAME,MOBILE_NUM,PHONE_NUM,EMAIL_ADDR,BUS_PARTN_NUM,ADDR_HOUSE_NUM,...,METER_TYPE,CREATION_DATE,MODIFIED_DATE,METERREADSEQUENCE,CONSUMP18MOMONTHLYAVG,CONSUMP18MOMONTHLYMAX,CONTRACT_START,ORGANIZATION,METER_READ_DATE,FULLADDRESS
0,136749,5000006879,ELEC,01,KENNETH R. EHRESMANN,603-661-0358,None,KENNYEHRESMANN@HOTMA,1000233502,1203,...,ES1C3C,1900-01-01,NaT,00000240,218.0,576.0,20180814,None,2016-07-06 16:33:18,"1203 NW 16TH AVE, 4, GAINESVILLE"
1,136753,5000006885,ELEC,01,KENNETH R. EHRESMANN,603-661-0358,None,KENNYEHRESMANN@HOTMA,1000233502,1203,...,ES1C3C,1900-01-01,NaT,00000250,224.0,575.0,20161021,None,2016-07-06 16:33:18,"1203 NW 16TH AVE, 3, GAINESVILLE"
2,5972,5000018516,ELEC,01,GERTRUDE R. LINDLER,None,352-375-2003,None,1000010957,1024,...,ES1C3C,1900-01-01,NaT,00000490,103.0,137.0,20070404,None,2016-07-06 16:33:20,"1024 NW 15TH AVE, GAINESVILLE"
3,6168,5000019041,ELEC,01,JONATHAN A. VAROL,352-226-4472,None,JVAROL@GMAIL.COM,1000194280,1423,...,ES1C3C,1900-01-01,NaT,00002410,823.0,1417.0,20170310,None,2016-07-05 16:32:04,"1423 NW 8TH ST, GAINESVILLE"
4,6177,5000019076,ELEC,01,GEORGE EMMANUEL,None,352-378-5172,GEMMANUEL@BELLSOUTH.,1000015514,723,...,ES3C4C,1900-01-01,NaT,00002490,219.0,305.0,20070404,None,2015-02-03 17:03:34,"723 NW 16TH AVE, GAINESVILLE"


In [43]:
df.to_csv(ivr_list,header=True, index=True)